In [3]:
import kagglehub
kagglehub.login()

In [4]:
yuvalsamoilov_danenberg2022_path = kagglehub.dataset_download('yuvalsamoilov/danenberg2022')
print('Data source import complete.')


Data source import complete.


In [5]:
!pip install pytorch_msssim
!pip install imagecodecs
!pip install torchmetrics
!pip install clean-fid
!pip install optuna

In [6]:
import os
import tifffile
import zipfile
import imagecodecs
import tempfile
import pickle
import json
import imageio
import sys
import psutil
import shutil
import numpy as np
import pandas as pd
import gc
import time
import glob
from collections import Counter, OrderedDict
import cv2
import random
from scipy.stats import pearsonr
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.parametrizations import weight_norm
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from torch.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision import models
import skimage.transform
from pytorch_msssim import ms_ssim
from tqdm import tqdm
import torchvision.utils as vutils
from cleanfid import fid
import optuna
from optuna.trial import Trial
#from torchmetrics.image.fid import FrechetInceptionDistance


import matplotlib.pyplot as plt
params = {'axes.titlesize': 30,
          'legend.fontsize': 16,
          'figure.figsize': (16, 10),
          'axes.labelsize': 16,
          'axes.titlesize': 12,
          'xtick.labelsize': 16,
          'ytick.labelsize': 16,
          'figure.titlesize': 30}

plt.rcParams.update(params)
plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
INPUT_CACHE_ROOT = "/your-notebook-output-files/cv_cache"
WORK_CACHE_ROOT  = "/cv_cache"

# restore previous cache if present
if os.path.exists(INPUT_CACHE_ROOT):
    os.makedirs(WORK_CACHE_ROOT, exist_ok=True)
    for src in glob.glob(os.path.join(INPUT_CACHE_ROOT, "**/*"), recursive=True):
        if os.path.isfile(src):
            dst = src.replace(INPUT_CACHE_ROOT, WORK_CACHE_ROOT, 1)
            os.makedirs(os.path.dirname(dst), exist_ok=True)
            shutil.copy2(src, dst)
    print(f"[init] Restored cache from {INPUT_CACHE_ROOT} -> {WORK_CACHE_ROOT}")
else:
    os.makedirs(WORK_CACHE_ROOT, exist_ok=True)
    print("No previous cache found; starting fresh.")

No previous cache found; starting fresh.


In [6]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            52Gi       1.3Gi        29Gi       1.0Mi        22Gi        51Gi
Swap:             0B          0B          0B


In [11]:
class ResNet50Encoder(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        backbone = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = backbone.bn1
        self.relu = backbone.relu
        self.maxpool = backbone.maxpool
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4
        for param in backbone.parameters():
            param.requires_grad = False
            
        for param in self.layer2.parameters():
            param.requires_grad = True
            
        for param in self.layer3.parameters():
            param.requires_grad = True
            
        for param in self.layer4.parameters():
            param.requires_grad = True

    def forward(self, x):
        f0 = self.conv1(x)
        f0 = self.bn1(f0)
        f0 = self.relu(f0)
        f0_pool = self.maxpool(f0)
        f1 = self.layer1(f0_pool)
        f2 = self.layer2(f1)
        f3 = self.layer3(f2)
        f4 = self.layer4(f3)
        return f0, f1, f2, f3, f4


class UNetDecoder(nn.Module):
    def __init__(self, out_channels=1, p=0.2):
        super().__init__()
        # Stage f4(2048) -> f3 scale (target 1024 ch)
        self.conv4_1 = nn.Sequential(
            nn.Conv2d(2048 + 1024, 1024, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p),
        )
        self.conv4_2 = nn.Sequential(
            nn.Conv2d(1024, 1024, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
        )

        # Stage -> f2 scale (target 512 ch)
        self.conv3_1 = nn.Sequential(
            nn.Conv2d(1024 + 512, 512, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p),
        )
        self.conv3_2 = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
        )

        # Stage -> f1 scale (target 256 ch)
        self.conv2_1 = nn.Sequential(
            nn.Conv2d(512 + 256, 256, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p),
        )
        self.conv2_2 = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        # Stage -> combine (f1 + f0) at f1 scale (target 64 ch)
        self.conv1_1 = nn.Sequential(
            nn.Conv2d(256 + 256 + 64, 64, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p),
        )
        self.conv1_2 = nn.Sequential(
            nn.Conv2d(64, 64, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )

        # Final upsample 
        self.final_up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)

        # Light head
        self.up0 = nn.Sequential(
            nn.Conv2d(64, 32, 3, padding=1, padding_mode='reflect', bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )
        self.final = weight_norm(nn.Conv2d(32, out_channels, 1))

    def forward(self, skips):
        f0, f1, f2, f3, f4 = skips 

        # ---- up4: f4 -> f3 scale ----
        x = F.interpolate(f4, scale_factor=2, mode='bilinear', align_corners=False)
        if x.shape[-2:] != f3.shape[-2:]:
            f3 = F.interpolate(f3, size=x.shape[-2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, f3], dim=1)
        x = self.conv4_1(x)
        x = self.conv4_2(x)

        # ---- up3: -> f2 scale ----
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        if x.shape[-2:] != f2.shape[-2:]:
            f2 = F.interpolate(f2, size=x.shape[-2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, f2], dim=1)
        x = self.conv3_1(x)
        x = self.conv3_2(x)

        # ---- up2: -> f1 scale ----
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)
        if x.shape[-2:] != f1.shape[-2:]:
            f1 = F.interpolate(f1, size=x.shape[-2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, f1], dim=1)
        x = self.conv2_1(x)
        x = self.conv2_2(x)

        # ---- up1: combine f1 & f0 at f1 scale ----
        # f0 may need resize to f1 scale
        if f0.shape[-2:] != f1.shape[-2:]:
            f0 = F.interpolate(f0, size=f1.shape[-2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, f1, f0], dim=1)
        x = self.conv1_1(x)
        x = self.conv1_2(x)

        # ---- final upsample + head ----
        x = self.final_up(x) 
        x = self.up0(x)
        x = F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False) 
        x = torch.sigmoid(self.final(x))
        return x

class ResNetUNet(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()
        self.encoder = ResNet50Encoder(in_channels=in_channels)
        self.decoder = UNetDecoder()

    def forward(self, x):
        skips = self.encoder(x)
        return self.decoder(skips)

# Wrap the lists in a custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, file_list, target_idx, control_markers_indices,
                 regression_models, pca, pca_mean, pca_std, input_channels,
                 n_components=3, cofactor=5.0, chunk_size=100_000):
        self.file_list = file_list
        self.target_idx = target_idx
        self.control_markers_indices = control_markers_indices
        self.regression_models = regression_models
        self.pca = pca
        self.pca_mean = pca_mean
        self.pca_std = pca_std
        self.input_channels = input_channels
        self.n_components = n_components
        self.cofactor = cofactor
        self.chunk_size = chunk_size

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        x_tensor, y_tensor, _ = preprocess_image(
            file_path,
            self.target_idx,
            self.control_markers_indices,
            self.regression_models,
            self.pca,
            self.pca_mean,
            self.pca_std,
            self.input_channels,
            self.n_components,
            self.cofactor,
            self.chunk_size
        )
        H, W = x_tensor.shape[1], x_tensor.shape[2]  
        return x_tensor, y_tensor, (H, W)

In [12]:
def to_hwc(img):
    '''
    Different libraries use different conventions.
    - PyTorch: prefers CHW format (channels, height, width)
    - TensorFlow/PIL/Matplotlib: prefer HWC format (height, width, channels)
    The function ensures all images are consistently in HWC format for the preprocessing pipeline, regardless of how they were originally stored or loaded.
    '''
    if img.ndim == 2:
        img = img[..., None]
    if img.shape[0] < img.shape[1] and img.shape[0] < img.shape[2]:
        img = np.transpose(img, (1, 2, 0))
    return img.astype(np.float32, copy=False)

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def fit_preprocessing(
    train_files,
    target_idx,
    control_markers_indices,
    n_components=3,
    batch_size=100_000,
    cofactor=5.0,
    eps=1e-8):
    """
    Fit channel-wise regressions and incremental PCA on residuals from IMC training images with lazy loading.
    Returns: (regression_models, pca, pca_mean, pca_std, input_channels)
    """

    # Discover channel setup from the first image
    input_channels = []
    img0 = to_hwc(tifffile.imread(train_files[0]))
    #print(f"[info] example image shape: {img0.shape}")
    H, W, C = img0.shape
    input_channels = [c for c in range(C) if c != target_idx and c not in control_markers_indices]
    K = len(input_channels)
    del img0; gc.collect()

    if not input_channels:
        print(f"Warning: no inputs for target {target_idx} with controls {control_markers_indices}.")
        return None, None, None, None, input_channels

    # 1. Fit regressions with SGDRegressor (partial_fit: true "streaming" mode)
    regression_models = {}
    first_pass = {}

    for j in input_channels:
        regression_models[j] = SGDRegressor(max_iter=1000, tol=1e-3)
        first_pass[j] = True

    for c, f in enumerate(train_files):
        if c % int(len(train_files)/2) == 0 or c == len(train_files)-1:
            print(f"[prepro] extracting control and trg markers: {c+1} / {len(train_files)}")
        img = to_hwc(tifffile.imread(f))
        Xc = img[..., control_markers_indices].reshape(-1, len(control_markers_indices))
        for j in input_channels:
            y = img[..., j].reshape(-1)
            if first_pass[j]:
                regression_models[j].partial_fit(Xc, y)
                first_pass[j] = False
            else:
                regression_models[j].partial_fit(Xc, y)
        del img, Xc, y; gc.collect()

    # 2. Incremental PCA on residuals (streamed, as before)
    if n_components > K:
        print(f"n_components {n_components} > residual dim {K}; using {K}.")
        n_components = K
    pca = IncrementalPCA(n_components=n_components)

    for c, f in enumerate(train_files):
        if c % int(len(train_files)/2) == 0 or c == len(train_files)-1:
            print(f"[prepro] fitting regression: {c+1} / {len(train_files)}")

        img = to_hwc(tifffile.imread(f))
        Xc = img[..., control_markers_indices].reshape(-1, len(control_markers_indices))
        R = np.empty((Xc.shape[0], K), dtype=np.float32)
        for k, j in enumerate(input_channels):
            y = img[..., j].reshape(-1)
            y_pred = regression_models[j].predict(Xc)
            R[:, k] = y - y_pred.astype(np.float32, copy=False)
        R = np.arcsinh(R / cofactor).astype(np.float32)
        for s in range(0, R.shape[0], batch_size):
            batch = R[s : s + batch_size]
            pca.partial_fit(batch)
        del img, Xc, R; gc.collect()

    # 3. Compute mean/std of PCA space (streamed)
    total = 0
    sum_z = np.zeros(n_components, dtype=np.float64)
    sumsq_z = np.zeros(n_components, dtype=np.float64)

    for c, f in enumerate(train_files):
        if c % int(len(train_files)/2) == 0 or c == len(train_files)-1:
            print(f"[prepro] normalization: {c+1} / {len(train_files)}")

        img = to_hwc(tifffile.imread(f))
        Xc = img[..., control_markers_indices].reshape(-1, len(control_markers_indices))
        R = np.empty((Xc.shape[0], K), dtype=np.float32)
        for k, j in enumerate(input_channels):
            y = img[..., j].reshape(-1)
            y_pred = regression_models[j].predict(Xc)
            R[:, k] = y - y_pred.astype(np.float32, copy=False)
        R = np.arcsinh(R / cofactor).astype(np.float32)
        for s in range(0, R.shape[0], batch_size):
            Z = pca.transform(R[s : s + batch_size]).astype(np.float32)
            sum_z += Z.sum(axis=0)
            sumsq_z += (Z * Z).sum(axis=0)
            total += Z.shape[0]
        del img, Xc, R, Z; gc.collect()

    if total == 0:
        print("Warning: no residuals seen for PCA stats; using zeros/ones.")
        pca_mean = np.zeros(n_components, dtype=np.float32)
        pca_std = np.ones(n_components, dtype=np.float32) * eps
    else:
        pca_mean = (sum_z / total).astype(np.float32)
        var = (sumsq_z / total) - (pca_mean.astype(np.float64) ** 2)
        pca_std = (np.sqrt(np.maximum(var, 0.0)) + eps).astype(np.float32)

    # Print variance captured by PCs if possible
    if hasattr(pca, "explained_variance_ratio_") and pca.explained_variance_ratio_ is not None:
        pct = pca.explained_variance_ratio_[:n_components].sum() * 100.0
        print(f"[norm] PCA projection to {n_components}D captures {pct:.2f}% variance")

    return regression_models, pca, pca_mean, pca_std, input_channels

def preprocess_image(
    file_path,
    target_idx,
    control_markers_indices,
    regression_models,
    pca,
    pca_mean,
    pca_std,
    input_channels,
    n_components=3,
    cofactor=5.0,
    chunk_size=100_000):
    """
    Load and preprocess a single IMC image lazily by:
    - loading image,
    - normalizing target,
    - computing residuals from regression,
    - arcsinh transform,
    - PCA transform in batches,
    - Z-score normalization of PCA features,
    - return PyTorch tensors for model input.
    """

    img = tifffile.imread(file_path)
    img = to_hwc(img)
    H, W, C = img.shape
    filename = os.path.basename(file_path)

    # Normalize target channel intensities with arcsinh and min-max
    target_data = img[..., target_idx]
    y_arc = np.arcsinh(target_data / cofactor).astype(np.float32)
    y_min, y_max = y_arc.min(), y_arc.max()
    y_norm = (y_arc - y_min) / (y_max - y_min + 1e-8)
    y_tensor = torch.from_numpy(y_norm[None, ...]).float()  # (1, H, W)

    # Prepare control channel matrix
    n_ctrl = len(control_markers_indices)
    K = len(input_channels)
    Xc = img[..., control_markers_indices].reshape(-1, n_ctrl)           # (N, n_ctrl)
    Yin = np.stack([img[..., j].reshape(-1) for j in input_channels], 1)  # (N, K)

    # Stack regression coef and intercept for vectorized prediction
    B = np.stack([regression_models[j].coef_ for j in input_channels], axis=-1).astype(np.float32)  # (n_ctrl, K)
    b = np.array([regression_models[j].intercept_ for j in input_channels], dtype=np.float32)       # (K,)
    b = b.flatten()

    # Predict residuals Y - regression(X)
    Yhat = Xc @ B + b                                                # (N, K)
    R = Yin - Yhat
    R = np.arcsinh(R / cofactor).astype(np.float32)

    # PCA transform in chunks, for low memory
    N = R.shape[0]
    Z = np.empty((N, n_components), dtype=np.float32)
    for s in range(0, N, chunk_size):
        e = s + chunk_size
        Z[s:e] = pca.transform(R[s:e])

    # Z-score normalize PCA components
    Z = (Z - pca_mean) / (pca_std + 1e-8)
    Z = Z.reshape(H, W, n_components)

    # Convert PCA tensor to channel-first for PyTorch
    x_tensor = torch.from_numpy(Z).permute(2, 0, 1).contiguous().float()  # (C, H, W)

    return x_tensor, y_tensor, filename

def apply_preprocessing(
    files,
    target_idx,
    control_markers_indices,
    regression_models,
    pca,
    pca_mean,
    pca_std,
    input_channels,
    n_components=3,
    cofactor=5.0,
    chunk_size=100_000):
    
    X_pca_tensor, Y_norm_list, filenames = [], [], []

    for c, f in enumerate(files):
        if c % int(len(files)/2) == 0 or c == len(files)-1:
            print(f"[norm] applying preprocessing: {c+1} / {len(files)}")

        x_tensor, y_tensor, filename = preprocess_image(
            f, target_idx, control_markers_indices, regression_models, pca,
            pca_mean, pca_std, input_channels, n_components, cofactor, chunk_size
        )
        X_pca_tensor.append(x_tensor)
        Y_norm_list.append(y_tensor)
        filenames.append(filename)
    return X_pca_tensor, Y_norm_list, filenames

def pad_collate(batch):
    """
    Pads all tensors in a batch to the largest size.
    Also returns original sizes for cropping.
    """
    X_list, Y_list, original_sizes = zip(*batch)

    max_H = max([x.shape[1] for x in X_list])
    max_W = max([x.shape[2] for x in X_list])

    padded_X_list = []
    padded_Y_list = []

    for x_tensor, y_tensor in zip(X_list, Y_list):
        pad_H = max_H - x_tensor.shape[1]
        pad_W = max_W - x_tensor.shape[2]

        padded_X = F.pad(x_tensor, (0, pad_W, 0, pad_H))
        padded_Y = F.pad(y_tensor, (0, pad_W, 0, pad_H))

        padded_X_list.append(padded_X)
        padded_Y_list.append(padded_Y)


    return torch.stack(padded_X_list), torch.stack(padded_Y_list), original_sizes

def train_model(model, train_loader, val_loader, device, title,
                config,
                epochs=50,
                huber_beta=1.0,
                grad_clip=1.0,
                use_amp=True,
                patience=15,
                save_path="best_model.pth",
                dataset=None,
                ):

    """
    Loss = alpha * Huber(y_hat, y) + (1 - alpha) * (1 - SSIM(y_hat_norm, y_norm)).
    - Huber is on intensities.
    - SSIM is per-image min-max normalized to [0,1].
    """

    model.to(device)
    
    lr = config.get('lr')
    alpha = config.get('alpha')
    opt = torch.optim.Adam(model.parameters(), lr = lr, weight_decay=1e-5)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="min", factor=0.5, patience=3)
    scaler = GradScaler(enabled=use_amp)

    train_losses, val_losses = [], []
    best_val, best_state, best_epoch = float("inf"), None, None
    no_improve = 0
    epsilon = 1e-8

    for epoch in range(epochs):
        # Training loop unchanged
        model.train()
        running = 0.0
        for x, y, original_sizes in train_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            B, _, H_padded, W_padded = x.shape  # Padded dimensions

            opt.zero_grad(set_to_none=True)
            with autocast(device_type='cuda', enabled=use_amp):
                yhat = model(x)

                hub_sum = 0.0
                ssim_sum = 0.0
                skip_batch = False

                for i in range(B):
                    h, w = original_sizes[i]
                    y_i = y[i:i+1, :, :h, :w]  # Ground truth (cropped)
                    yhat_i = yhat[i:i+1, :, :h, :w]  # Prediction (cropped)
        
                    hub_i = F.smooth_l1_loss(yhat_i, y_i, beta=huber_beta, reduction="mean")
                    hub_sum += hub_i

                    y_min, y_max = y_i.min().detach(), y_i.max().detach()
                    yh_min, yh_max = yhat_i.min().detach(), yhat_i.max().detach()
                    y_n = (y_i - y_min) / (y_max - y_min + epsilon)
                    yhat_n = (yhat_i - yh_min) / (yh_max - yh_min + epsilon)
                    
                    ssim_sum += (1.0 - ms_ssim(yhat_n, y_n, data_range=1.0, win_size=7, weights=[0.4, 0.3, 0.3]))

                hub_mean = hub_sum / B
                ssim_mean = ssim_sum / B
                
                hub_mean = torch.as_tensor(hub_mean, device=device)
                ssim_mean = torch.as_tensor(ssim_mean, device=device)

                loss = alpha * hub_mean + (1 - alpha) * ssim_mean

                scaler.scale(loss).backward()
                if grad_clip is not None:
                    scaler.unscale_(opt)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                scaler.step(opt)
                scaler.update()

                running += loss.item() * B

        train_loss = running / len(train_loader.dataset)
        train_losses.append(train_loss)

        # Validation loop with robust error checking and batch skipping
        model.eval()
        val_running = 0.0
        valid_batches = 0
        with torch.no_grad(), autocast(device_type='cuda', enabled=use_amp):
            for batch_idx, (x, y, original_sizes) in enumerate(val_loader):
                x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
                B, _, H_padded, W_padded = x.shape  # Padded dimensions

                yhat = model(x)

                hub_sum = 0.0
                ssim_sum = 0.0
                skip_batch = False

                for i in range(B):
                    h, w = original_sizes[i]
                    y_i = y[i:i+1, :, :h, :w]
                    yhat_i = yhat[i:i+1, :, :h, :w]

                    # Clamp predictions between 0 and 1 for numerical stability
                    yhat_i = torch.clamp(yhat_i, min=0.0, max=1.0)
                        
                    y_min, y_max = y_i.min(), y_i.max()
                    yh_min, yh_max = yhat_i.min(), yhat_i.max()
                    denom_y = (y_max - y_min).abs() + epsilon
                    denom_yh = (yh_max - yh_min).abs() + epsilon

                    if denom_y < epsilon or denom_yh < epsilon:
                        #print(f"Note: validation batch {batch_idx}, sample {i} skipped due to near-constant values")
                        skip_batch = True
                        break

                    y_n = (y_i - y_min) / denom_y
                    yhat_n = (yhat_i - yh_min) / denom_yh

                    ssim_val = ms_ssim(yhat_n, y_n, data_range=1.0, win_size=7, weights=[0.4, 0.3, 0.3])
                    if torch.isnan(ssim_val) or torch.isinf(ssim_val):
                        #print(f"Note: validation batch {batch_idx}, sample {i} SSIM returned NaN or Inf, skipping batch")
                        skip_batch = True
                        break

                    hub_i = F.smooth_l1_loss(yhat_i, y_i, beta=huber_beta, reduction="mean")
                    if torch.isnan(hub_i) or torch.isinf(hub_i):
                        #print(f"Note: validation batch {batch_idx}, sample {i} Huber loss returned NaN or Inf, skipping batch")
                        skip_batch = True
                        break

                    hub_sum += hub_i
                    ssim_sum += (1.0 - ssim_val)

                if skip_batch:
                    continue  # Skip entire batch aggregation

                val_loss_batch = (alpha * hub_sum + (1 - alpha) * ssim_sum) / B
                if torch.isnan(val_loss_batch) or torch.isinf(val_loss_batch):
                    #print(f"Note: validation batch {batch_idx} computed val_loss_batch as NaN or Inf, skipping")
                    continue

                val_running += float(val_loss_batch) * B
                valid_batches += B

        if valid_batches > 0:
            val_loss = val_running / valid_batches
        else:
            val_loss = float('nan')

        val_losses.append(val_loss)
        sched.step(val_loss)
        
        print(f"Epoch {epoch+1:>3d} | Train {train_loss:.3f} | Val {val_loss:.3f}")

        # ---- Early stopping ----
        if val_loss + 1e-6 < best_val:
            best_val, best_epoch = val_loss, epoch + 1
            best_state = {k: v.clone() for k, v in model.state_dict().items()}
            no_improve = 0
            print(f"  ↳ New best val={val_loss:.3f} at epoch {best_epoch}")
            if save_path:
                torch.save(best_state, save_path)
                print(f"  ↳ Saved checkpoint to {save_path}")
        else:
            no_improve += 1
            print(f"  ↳ No improvement ({no_improve}/{patience} patience)")
            current_lr = opt.param_groups[0]["lr"]
            print(f"  ↳ LR: {current_lr:.2e}")
            if patience is not None and no_improve >= patience:
                print(f"--Early stopping at epoch {epoch+1}. Best epoch was {best_epoch} with val={best_val:.3f}")
                break

    # Restore best weights
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"--Restored model weights from epoch {best_epoch} (val={best_val:.3f})")

    # Plot losses
    plt.figure(figsize=(8, 5))
    plt.plot(range(1, len(train_losses)+1), train_losses, marker='o', label='Train')
    plt.plot(range(1, len(val_losses)+1), val_losses, marker='s', label='Val')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title(f'Marker: {title}')
    plt.legend(); plt.grid(True); plt.show()

    return train_losses, val_losses
    
def visualize_prediction(model, dataset, device, indices=None, target_channel=0, name="comparison_grid"):
    """
    Show & save some example pairs (GT | Pred). 
    """

    model = model.to(device)
    model.eval()
    eps = 1e-8  # small constant to avoid division by zero

    if indices is None:
        k = 25
        # sample without replacement; if len<5 just take range(k)
        indices = random.sample(range(len(dataset)), k) if len(dataset) >= k else list(range(len(dataset)))

    # build figure
    n = len(indices)
    fig, axs = plt.subplots(n, 2, figsize=(8, 4 * n))
    if n == 1:
        axs = np.expand_dims(axs, axis=0)  # make it 2D

    with torch.no_grad():
        for row, idx in enumerate(indices):
            x_tensor, y_tensor, _ = dataset[idx]
            x_tensor = x_tensor.unsqueeze(0).to(device)
            y_tensor = y_tensor.unsqueeze(0).to(device)

            y_pred = model(x_tensor)

            # crop out tile borders to remove padding artifacts
            crop_margin = 64  # try 32 or 64 depending on your tile size
            _, _, H, W = y_pred.shape
            y_pred = y_pred[:, :, crop_margin:H - crop_margin, crop_margin:W - crop_margin]
            
            # also crop ground truth to match
            y_tensor = y_tensor[:, :, crop_margin:H - crop_margin, crop_margin:W - crop_margin]
            
            
            # to numpy, select channel, [0,1] normalize
            y_true_np = y_tensor.squeeze().detach().cpu().numpy()
            y_pred_np = y_pred.squeeze().detach().cpu().numpy()
            if y_true_np.ndim == 3:
                y_true_np = y_true_np[target_channel]
            if y_pred_np.ndim == 3:
                y_pred_np = y_pred_np[target_channel]
            
            # ----- visualization-only rescaling -----
            # Limitation: 
            # This visualization only rescales the predictions to look more like the ground truth 
            # it does not fix or reflect the true quantitative errors or background bias of the model (maybe add mask and replace ms_ssim with BCE).

            gt = y_true_np.astype(np.float32)
            pr = y_pred_np.astype(np.float32)
            gt_mean, gt_std = gt.mean(), gt.std()
            pr_mean, pr_std = pr.mean(), pr.std()
            pr_adj = (pr - pr_mean) / pr_std * gt_std + gt_mean
            stack = np.concatenate([gt.flatten(), pr_adj.flatten()])
            s_min, s_max = stack.min(), stack.max()
            s_range = s_max - s_min + eps
            
            y_true_vis = (gt - s_min) / s_range
            y_pred_vis = (pr_adj - s_min) / s_range

            #edits#
            vmin = y_true_np.min()
            vmax = y_true_np.max()
            #end#

            axs[row, 0].imshow(y_true_vis, cmap="inferno", vmin=vmin, vmax=vmax)
            axs[row, 0].set_title(f"Ground Truth (idx {idx})")
            axs[row, 0].axis("off")

            axs[row, 1].imshow(y_pred_vis, cmap="inferno", vmin=vmin, vmax=vmax)
            axs[row, 1].set_title("Prediction")
            axs[row, 1].axis("off")

    fig.suptitle(f"{name}", fontsize=14)
    fig.tight_layout(rect=[0, 0, 1, 0.97])

    # always save (headless-friendly), then show if a display exists
    out_path = f"comparison_grid_{name}.png"
    fig.savefig(out_path, dpi=400, bbox_inches="tight")
    plt.show() 
    plt.close(fig)

    print(f"[vis] Saved comparison figure to: {out_path}")

def smart_load_state_dict(model, state, strict=True):
    
    is_dp = isinstance(model, torch.nn.DataParallel)
    has_module = all(k.startswith("module.") for k in state.keys())
    if is_dp and not has_module:
        return model.module.load_state_dict(state, strict=strict)          
    if (not is_dp) and has_module:
        state = {k.replace("module.", "", 1): v for k, v in state.items()}  
        return model.load_state_dict(state, strict=strict)
    target = model.module if is_dp else model
    return target.load_state_dict(state, strict=strict)

def train_model_with_config(
    model, train_loader, val_loader, device, title,
    config: dict,
    save_path="best_model.pth"):
    """
    Modified training function that accepts a config dictionary.
    Returns best validation loss for optimization.
    """

    model.to(device)

    # Extract hyperparameters from config
    lr = config.get('lr', 1e-4)
    alpha = config.get('alpha', 0.8)
    huber_beta = config.get('huber_beta', 1.0)
    weight_decay = config.get('weight_decay', 1e-5)
    grad_clip = config.get('grad_clip', 1.0)
    scheduler_patience = config.get('scheduler_patience', 3)
    scheduler_factor = config.get('scheduler_factor', 0.5)
    epochs = config.get('epochs', 50)
    early_stop_patience = config.get('early_stop_patience', 10)

    # Setup optimizer and scheduler
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(
        opt, mode="min", factor=scheduler_factor, patience=scheduler_patience
    )
    scaler = GradScaler(enabled=True)

    best_val = float("inf")
    no_improve = 0
    epsilon = 1e-8
    
    for epoch in range(epochs):
        # Training
        model.train()
        running = 0.0
        for x, y, original_sizes in train_loader:
            x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
            B = x.shape[0]

            opt.zero_grad(set_to_none=True)
            with autocast(device_type='cuda', enabled=True):
                yhat = model(x)

                hub_sum = 0.0
                ssim_sum = 0.0
                for i in range(B):
                    h, w = original_sizes[i]
                    y_i = y[i:i+1, :, :h, :w]
                    yhat_i = yhat[i:i+1, :, :h, :w]

                    hub_i = F.smooth_l1_loss(yhat_i, y_i, beta=huber_beta, reduction="mean")
                    hub_sum += hub_i

                    y_min, y_max = y_i.min().detach(), y_i.max().detach()
                    yh_min, yh_max = yhat_i.min().detach(), yhat_i.max().detach()
                    y_n = (y_i - y_min) / (y_max - y_min + epsilon)
                    yhat_n = (yhat_i - yh_min) / (yh_max - yh_min + epsilon)
                    
                    ssim_sum += (1.0 - ms_ssim(yhat_n, y_n, data_range=1.0, win_size=7, weights=[0.4, 0.3, 0.3]))

                hub_mean = hub_sum / B
                ssim_mean = ssim_sum / B

                hub_mean = torch.as_tensor(hub_mean, device=device)
                ssim_mean = torch.as_tensor(ssim_mean, device=device)

                loss = alpha * hub_mean + (1 - alpha) * ssim_mean

            scaler.scale(loss).backward()
            if grad_clip is not None:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            scaler.step(opt)
            scaler.update()
            running += loss.item() * B

        train_loss = running / len(train_loader.dataset)

        # Validation
        model.eval()
        val_running = 0.0
        valid_batches = 0
        
        with torch.no_grad(), autocast(device_type='cuda', enabled=True):
            for x, y, original_sizes in val_loader:
                x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
                B = x.shape[0]
                yhat = model(x)

                hub_sum = 0.0
                ssim_sum = 0.0
                skip_batch = False

                for i in range(B):
                    h, w = original_sizes[i]
                    y_i = y[i:i+1, :, :h, :w]
                    yhat_i = yhat[i:i+1, :, :h, :w]
                    yhat_i = torch.clamp(yhat_i, min=0.0, max=1.0)

                    y_min, y_max = y_i.min(), y_i.max()
                    yh_min, yh_max = yhat_i.min(), yhat_i.max()
                    
                    y_n = (y_i - y_min) / ((y_max - y_min).abs() + epsilon)
                    yhat_n = (yhat_i - yh_min) / ((yh_max - yh_min).abs() + epsilon)

                    ssim_val = ms_ssim(yhat_n, y_n, data_range=1.0, weights=[0.4, 0.3, 0.3])
                                        # Check for NaN/Inf
                    if torch.isnan(ssim_val) or torch.isinf(ssim_val):
                        skip_batch = True
                        break
                    
                    hub_i = F.smooth_l1_loss(yhat_i, y_i, beta=huber_beta, reduction="mean")

                    if torch.isnan(hub_i) or torch.isinf(hub_i):
                        skip_batch = True
                        break
                        
                    hub_sum += hub_i
                    ssim_sum += (1.0 - ssim_val)

                if not skip_batch:
                    val_loss_batch = (alpha * hub_sum + (1 - alpha) * ssim_sum) / B
                    if not (torch.isnan(val_loss_batch) or torch.isinf(val_loss_batch)):
                        val_running += float(val_loss_batch) * B
                        valid_batches += B

        val_loss = val_running / valid_batches if valid_batches > 0 else float('nan')
        sched.step(val_loss)

        # Early stopping
        if val_loss + 1e-6 < best_val:
            best_val = val_loss
            no_improve = 0
            if save_path:
                torch.save(model.state_dict(), save_path)
        else:
            no_improve += 1
            if no_improve >= early_stop_patience:
                print(f"Early stop at epoch {epoch+1}, best val={best_val:.4f}")
                break

        print(f"Epoch {epoch+1}/{epochs} | Train: {train_loss:.4f} | Val: {val_loss:.4f}")

    return best_val

def objective(trial: Trial, train_loader, val_loader, device, model_fn, title):
    """
    Optuna objective function for hyperparameter search.

    Args:
        trial: Optuna trial object
        train_loader: Training data loader
        val_loader: Validation data loader
        device: PyTorch device
        model_fn: Function that returns a fresh model instance
        title: Experiment title

    Returns:
        best_val_loss: Best validation loss achieved
    """

    # Sample hyperparameters
    config = {
        'lr': trial.suggest_float('lr', 3e-6, 5e-4, log=True),
        'alpha': trial.suggest_float('alpha', 0.9, 0.99),
        'epochs': 5, 

    }

    # Create fresh model
    model = model_fn()

    # Train and get best validation loss
    try:
        best_val = train_model_with_config(
            model, train_loader, val_loader, device, title,
            config=config,
            save_path=f"trial_{trial.number}_best.pth"
        )
        return best_val
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return float('inf')

def run_hyperparameter_search(
    train_loader, 
    val_loader, 
    device, 
    model_fn,
    title,
    n_trials=6,
    study_name="stain_translation_opt"):
    """
    Run Optuna hyperparameter optimization.

    Args:
        train_loader: Training data loader
        val_loader: Validation data loader
        device: PyTorch device
        model_fn: Function that returns a fresh model instance (e.g., lambda: ResNet50UNet(...))
        title: Experiment title
        n_trials: Number of optimization trials
        study_name: Name for the Optuna study

    Returns:
        study: Optuna study object with results
    """

    # Create study
    study = optuna.create_study(
        direction='minimize',
        study_name=study_name,
        sampler=optuna.samplers.TPESampler(seed=42),
    )

    # Run optimization
    study.optimize(
        lambda trial: objective(trial, train_loader, val_loader, device, model_fn, title),
        n_trials=n_trials,
        show_progress_bar=False
    )

    # Print results
    print("\n" + "="*10)
    print("HYPERPARAMETER OPTIMIZATION RESULTS")
    print("="*10)
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best validation loss: {study.best_value:.6f}")
    print(f"\nBest hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")

    # Plot optimization history
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

    return study

def optimize_for_marker(path, target_idx, control_markers_indices, title, 
                        data_name, sample_fraction=0.2): 
    """Run optimization on fold 1, return best hyperparameters."""
    
    n_components = 3
    cofactor = 5.0
    batch_size = 2
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    all_files = sorted([os.path.join(path, f) for f in os.listdir(path) 
                       if f.endswith(('.tif', '.tiff'))])
    
    random.shuffle(all_files)
    split_idx = int(0.75 * len(all_files))
    train_files = all_files[:split_idx]
    val_files = all_files[split_idx:]
    
    print(f"[split] {len(train_files)} train, {len(val_files)} val")

    # Fit preprocessing
    print(f"[fitting] Running preprocessing...")
    regression_models, pca, pca_mean, pca_std, input_channels = fit_preprocessing(
        train_files, target_idx, control_markers_indices, 
        n_components=n_components, cofactor=cofactor
    )
    
    # Subsample training data (for speed)
    n_train_samples = max(50, int(len(train_files) * sample_fraction))
    train_files_subset = random.sample(train_files, n_train_samples)
    
    print(f"[info] Using {len(train_files_subset)} train samples (20%)")
    print(f"[info] Using {len(val_files)} val samples (100%)")

    # Apply preprocessing
    X_train, Y_train, _ = apply_preprocessing(
        train_files_subset,  
        target_idx, control_markers_indices,
        regression_models, pca, pca_mean, pca_std, input_channels,
        n_components=n_components, cofactor=cofactor
    )
    
    X_val, Y_val, _ = apply_preprocessing(
        val_files,
        target_idx, control_markers_indices,
        regression_models, pca, pca_mean, pca_std, input_channels,
        n_components=n_components, cofactor=cofactor
    )
    
    train_dataset = list(zip(X_train, Y_train, [x.shape[1:] for x in X_train]))
    val_dataset = list(zip(X_val, Y_val, [x.shape[1:] for x in X_val]))
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                             shuffle=True, collate_fn=pad_collate)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, 
                           shuffle=False, collate_fn=pad_collate)
    
    # Optimization
    def create_model():
        return ResNetUNet(in_channels=n_components)
    
    study = run_hyperparameter_search(
        train_loader, val_loader, device, create_model,
        title=title, n_trials=6, study_name=f"{data_name}_{title}_opt"
    )
    
    return study.best_params

In [13]:
def main(
    path: str,
    target_idx: int,
    control_markers_indices: list,
    title: str = "Unspecified",
    n_splits: int = 3,
    batch_size: int = 2,
    device: str = None,
    data_name: str = None,
    cache_dir: str = "./cv_cache",
    resume: bool = True,
    n_components: int = 3,
    cofactor: float = 5.0,
    n_trials: int = 10,
    finetune = False
):
    '''
    Complete workflow:
    1. Split data into train (80%) and test (20%)
    2. Run hyperparameter optimization with k-fold CV on train set only
    3. Train final model on full train set with best hyperparameters
    4. Evaluate on held-out test set
    '''

    os.makedirs(cache_dir, exist_ok=True)
    set_seed(42)

    best_hyperparams = {'lr': 1e-3,
                        'alpha': 0.90}
    
        
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    all_files = sorted([
        os.path.join(path, f) for f in os.listdir(path) 
        if f.endswith(('.tif', '.tiff'))
    ])

    split_path = os.path.join(cache_dir, f"train_test_split_{data_name}_{title}.pkl")
    if resume and os.path.exists(split_path):
        with open(split_path, "rb") as f:
            train_files, test_files = pickle.load(f)
        print(f"[init] Loaded train/test split from {split_path}")
    else:
        random.shuffle(all_files)
        split_idx = int(0.75 * len(all_files))
        train_files = all_files[:split_idx]
        test_files = all_files[split_idx:]
        with open(split_path, "wb") as f:
            pickle.dump((train_files, test_files), f)
        print(f"[init] Created train/test split: {len(train_files)} train, {len(test_files)} test")

    print(f"[split] Train: {len(train_files)}, Test: {len(test_files)}")

    # STEP 2: Hyperparameter Optimization with CV on Train Set
    #print(f"\n{'='*10}")
    #print(f"STEP 2: HYPERPARAMETER OPTIMIZATION")
    #print(f"{'='*10}")
    
    
    if finetune == True:
        cv_files = random.sample(train_files, len(train_files)//3) #third of train for cv and hyperparams finetuning 

        best_hyperparams = run_cv_optimization(
            train_files=cv_files,
            target_idx=target_idx,
            control_markers_indices=control_markers_indices,
            title=title,
            data_name=data_name,
            n_splits=n_splits,
            batch_size=batch_size,
            device=device,
            cache_dir=cache_dir,
            n_components=n_components,
            cofactor=cofactor,
            n_trials=n_trials,
            resume=resume
        )

        #print(f"\n[result] Best hyperparameters found:")
        #print(f"  lr: {best_hyperparams['lr']:.4f}")
        #print(f"  alpha: {best_hyperparams['alpha']:.4f}")

        # Save best hyperparameters
        hyperparam_path = os.path.join(cache_dir, f"best_hyperparams_{data_name}_{title}.json")
        with open(hyperparam_path, "w") as f:
            json.dump(best_hyperparams, f, indent=2)
        print(f"[cache] Saved best hyperparameters to {hyperparam_path}")

        print(f"\n{'='*10}")
        print(f"STEP 3: TRAINING FINAL MODEL (full train set)")
        print(f"{'='*10}")

    final_model_path = os.path.join(cache_dir, f"final_model_{data_name}_{title}.pth")
    preproc_path = os.path.join(cache_dir, f"final_preproc_{data_name}_{title}.pkl")

    # Fit preprocessing on full train set
    if resume and os.path.exists(preproc_path):
        with open(preproc_path, "rb") as f:
            reg_models, pca, pca_mean, pca_std, input_channels = pickle.load(f)
        print(f"[init] Loaded preprocessing from {preproc_path}")
    else:
        print(f"[prepro] Fitting on {len(train_files)} training images...")
        reg_models, pca, pca_mean, pca_std, input_channels = fit_preprocessing(
            train_files, target_idx, control_markers_indices,
            n_components=n_components, cofactor=cofactor
        )
        with open(preproc_path, "wb") as f:
            pickle.dump((reg_models, pca, pca_mean, pca_std, input_channels), f)
        print(f"[cache] Saved preprocessing to {preproc_path}")

    # Create datasets
    train_dataset = CustomDataset(
        train_files, target_idx, control_markers_indices,
        reg_models, pca, pca_mean, pca_std, input_channels,
        n_components=n_components, cofactor=cofactor
    )

    test_dataset = CustomDataset(
        test_files, target_idx, control_markers_indices,
        reg_models, pca, pca_mean, pca_std, input_channels,
        n_components=n_components, cofactor=cofactor
    )

    train_size = int(0.75 * len(train_dataset))
    
    val_size = len(train_dataset) - train_size

    train_subset, val_subset = random_split(
        train_dataset, [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_loader = DataLoader(
        train_subset, batch_size=batch_size, shuffle=True,
        collate_fn=pad_collate, num_workers=2, pin_memory=True
    )
    val_loader = DataLoader(
        val_subset, batch_size=batch_size, shuffle=False,
        collate_fn=pad_collate, num_workers=2, pin_memory=True
    )

    # Train final model
    model = ResNetUNet(in_channels=n_components).to(device)
    
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    
    if resume and os.path.exists(final_model_path):
        print(f"[init] Loading existing final model from {final_model_path}")
        state = torch.load(final_model_path, map_location=device)
        smart_load_state_dict(model, state, strict=True)
    else:
        print(f"[training] Training final model")
        train_model(
            model, train_loader, val_loader, device,
            config=best_hyperparams,
            title=f"{title} (final)",
            epochs=50,
            patience=12,
            save_path=final_model_path,
            dataset=data_name
        )
        print(f"[cache] Saved final model to {final_model_path}")

    print(f"\n{'='*10}")
    print(f"FINAL EVALUATION")
    print(f"{'='*10}")

    test_metrics = evaluate_model(
        model=model,
        dataset=test_dataset,
        device=device,
        batch_size=batch_size,
        title=title,
        data_name=data_name
    )

    print(f"[FINAL RESULTS] {title}")
    print(f"  Test RMSE: {test_metrics['rmse']:.4f}")
    print(f"  Test Pearson: {test_metrics['pearson']:.4f}")
    print(f"  Test FID: {test_metrics['fid']:.2f}")
    print(f"  Hyperparams: {best_hyperparams}")

    visualize_prediction(
        model=model,
        dataset=test_dataset,
        device=device,
        indices=None,
        target_channel=0,
        name=title
    )

    # Save final metrics
    metrics_path = os.path.join(cache_dir, f"final_metrics_{data_name}_{title}.json")
    with open(metrics_path, "w") as f:
        json.dump(test_metrics, f, indent=2)
    print(f"[cache] Saved final metrics to {metrics_path}")

def run_cv_optimization(
    train_files,
    target_idx,
    control_markers_indices,
    title,
    data_name,
    n_splits=3,
    batch_size=2,
    device="cuda",
    cache_dir="./cv_cache",
    n_components=3,
    cofactor=5.0,
    n_trials=10,
    resume=True
):
    '''
    Run k-fold cross-validation with Optuna hyperparameter optimization.
    Uses ONLY train_files - no test set contamination.

    Returns:
        best_hyperparams: dict with optimal 'lr' and 'alpha'
    '''

    # Create k-fold splits on train files ONLY
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    cv_splits = []
    for tr_idx, va_idx in kf.split(train_files):
        tr_files = [train_files[i] for i in tr_idx]
        va_files = [train_files[i] for i in va_idx]
        cv_splits.append((tr_files, va_files))

    print(f"[cv] Created {n_splits}-fold CV splits on {len(train_files)} training files")

    # Define Optuna objective that uses CV
    def objective(trial: Trial):
        # Sample hyperparameters
        lr = trial.suggest_float('lr', 3e-5, 5e-4, log=True)
        alpha = trial.suggest_float('alpha', 0.9, 0.99)

        fold_val_losses = []

        # Evaluate this hyperparameter set across all CV folds
        for fold_idx, (tr_files, va_files) in enumerate(cv_splits, 1):
            print(f"[Trial {trial.number}] Fold {fold_idx}/{n_splits} | lr={lr:.2e}, alpha={alpha:.3f}")

            # Fit preprocessing on this fold's training data
            reg_models, pca, pca_mean, pca_std, input_channels = fit_preprocessing(
                tr_files, target_idx, control_markers_indices,
                n_components=n_components, cofactor=cofactor
            )

            # Create datasets
            tr_dataset = CustomDataset(
                tr_files, target_idx, control_markers_indices,
                reg_models, pca, pca_mean, pca_std, input_channels,
                n_components=n_components, cofactor=cofactor
            )
            va_dataset = CustomDataset(
                va_files, target_idx, control_markers_indices,
                reg_models, pca, pca_mean, pca_std, input_channels,
                n_components=n_components, cofactor=cofactor
            )

            tr_loader = DataLoader(tr_dataset, batch_size=batch_size, shuffle=True, 
                                   collate_fn=pad_collate)
            va_loader = DataLoader(va_dataset, batch_size=batch_size, shuffle=False,
                                   collate_fn=pad_collate)

            # Train model with these hyperparameters
            model = ResNetUNet(in_channels=n_components).to(device)
            if torch.cuda.device_count() > 1:
                model = torch.nn.DataParallel(model)

            config = {'lr': lr, 'alpha': alpha, 'epochs': n_trials}  # Shorter for speed

            try:
                val_loss = train_model_with_config(
                    model, tr_loader, va_loader, device, title,
                    config=config,
                    save_path=None  # Don't save intermediate models
                )
                fold_val_losses.append(val_loss)
            except Exception as e:
                print(f"  [Trial {trial.number}] Fold {fold_idx} failed: {e}")
                fold_val_losses.append(float('inf'))

            # Clean up
            del model, tr_dataset, va_dataset, tr_loader, va_loader
            torch.cuda.empty_cache()
            gc.collect()

        # Return mean validation loss across all folds
        mean_val_loss = np.mean(fold_val_losses)
        print(f"  [Trial {trial.number}] Mean CV loss: {mean_val_loss:.4f}")
        return mean_val_loss

    # Run Optuna optimization
    study = optuna.create_study(
        direction='minimize',
        study_name=f"{data_name}_{title}_cv_opt",
        sampler=optuna.samplers.TPESampler(seed=42)
    )

    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    # Print results
    print(f"\n{'='*10}")
    print(f"CV OPTIMIZATION COMPLETE")
    print(f"{'='*10}")
    print(f"Best trial: {study.best_trial.number}")
    print(f"Best CV loss: {study.best_value:.6f}")
    print(f"Best hyperparameters:")
    for key, value in study.best_params.items():
        print(f"  {key}: {value}")

    return study.best_params

def evaluate_model(model, dataset, device, batch_size, title, data_name):
    '''
    Evaluate model on a dataset and compute RMSE, Pearson, and FID metrics.
    '''
    model.eval()

    test_loader = DataLoader(
        dataset, batch_size=batch_size, shuffle=False,
        collate_fn=pad_collate, num_workers=2, pin_memory=True
    )

    rmse_list = []
    all_ypred_pixels, all_ytrue_pixels = [], []

    # Create temp directories for FID computation
    with tempfile.TemporaryDirectory() as pred_dir, tempfile.TemporaryDirectory() as true_dir:
        idx_img = 0

        with torch.no_grad(), autocast(device_type='cuda', enabled=True):
            for x, y, original_sizes in test_loader:
                x = x.to(device, non_blocking=True)
                y = y.to(device, non_blocking=True)

                yhat = model(x)

                B, _, H, W = y.shape
                yhat_cropped = yhat[:, :, :H, :W].clamp(0, 1)

                # RMSE
                mse_val = F.mse_loss(yhat_cropped, y).item()
                rmse_list.append(np.sqrt(mse_val))

                # Pearson - accumulate pixels
                yhat_flat = yhat_cropped.detach().cpu().numpy().ravel()
                y_flat = y.detach().cpu().numpy().ravel()
                all_ypred_pixels.append(yhat_flat)
                all_ytrue_pixels.append(y_flat)

                # FID - save normalized RGB versions
                if yhat_cropped.shape[1] == 1:
                    y_true_img = y.repeat(1, 3, 1, 1).float()
                    y_pred_img = yhat_cropped.repeat(1, 3, 1, 1).float()
                else:
                    y_true_img, y_pred_img = y.float(), yhat_cropped.float()

                # Per-image min-max to [0,1]
                for b in range(y_true_img.size(0)):
                    gt = y_true_img[b]
                    pr = y_pred_img[b]

                    gt = (gt - gt.min()) / (gt.max() - gt.min() + 1e-8)
                    pr = (pr - pr.min()) / (pr.max() - pr.min() + 1e-8)

                    gt_np = gt.permute(1, 2, 0).cpu().numpy()
                    pr_np = pr.permute(1, 2, 0).cpu().numpy()

                    # Resize to 512x512x3 and save uint8
                    gt_np = skimage.transform.resize(gt_np, (512, 512, 3), anti_aliasing=True, mode='reflect')
                    pr_np = skimage.transform.resize(pr_np, (512, 512, 3), anti_aliasing=True, mode='reflect')

                    gt_u8 = (gt_np * 255).clip(0, 255).astype(np.uint8)
                    pr_u8 = (pr_np * 255).clip(0, 255).astype(np.uint8)

                    imageio.imwrite(os.path.join(true_dir, f"{idx_img}.png"), gt_u8)
                    imageio.imwrite(os.path.join(pred_dir, f"{idx_img}.png"), pr_u8)
                    idx_img += 1

        # Compute FID
        fid_val = fid.compute_fid(pred_dir, true_dir, mode="clean", verbose=False)

    # Compute final metrics
    all_ypred_pixels = np.concatenate(all_ypred_pixels)
    all_ytrue_pixels = np.concatenate(all_ytrue_pixels)
    pearson_val, _ = pearsonr(all_ypred_pixels, all_ytrue_pixels)
    rmse_mean = float(np.mean(rmse_list))

    return {
        'marker': title,
        'rmse': rmse_mean,
        'pearson': float(pearson_val),
        'fid': float(fid_val)
    }

In [ ]:
dataset = "danenberg2022"
path = yuvalsamoilov_danenberg2022_path
control = [0,37,38] #histone_h3, dna1, dna2

marker_tasks = [
    ("ERa", 25, control),
    ("HER2_3b5", 12, control),
    ("HER2_d8f12", 19, control),
]

for idx, (title, target_idx, control_markers_indices) in enumerate(marker_tasks):
    start_time = time.time()
    print(f"\n{'='*10}")
    print(f"Processing: {title} (Marker {idx+1}/{len(marker_tasks)})")
    print(f"{'='*10}\n")
    
    result = main(
        path=path,
        target_idx=target_idx,
        control_markers_indices=control_markers_indices,
        title=title,
        data_name=dataset,
        cache_dir=WORK_CACHE_ROOT,
        n_splits=3,
        batch_size=2,
        n_components=3,
        cofactor=5.0,
        finetune = False
    )

    total_time = time.time() - start_time
    print(f"--Total training time for {title}: {total_time/60:.2f} minutes")



Processing: ERa (Marker 1/3)

[init] Loaded train/test split from /cv_cache/train_test_split_danenberg2022_ERa.pkl
[split] Train: 595, Test: 199
[init] Loaded preprocessing from /cv_cache/final_preproc_danenberg2022_ERa.pkl
[training] Training final model
Epoch   1 | Train 0.118 | Val 0.369
  ↳ New best val=0.369 at epoch 1
  ↳ Saved checkpoint to /cv_cache/final_model_danenberg2022_ERa.pth
Epoch   2 | Train 0.142 | Val 0.249
  ↳ New best val=0.249 at epoch 2
  ↳ Saved checkpoint to /cv_cache/final_model_danenberg2022_ERa.pth
